In [124]:
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
from prophet import Prophet
import keras
import datetime

In [80]:
PROCESSED_DIR = pathlib.Path("../data/processed/")
relevant_entries = ["Bad", "West", "Lene", "Stadion"]

In [49]:
analysis_df = pd.DataFrame()
for time_series_df_path in PROCESSED_DIR.glob("*.csv"):
    if "Bad" in str(time_series_df_path):
        df = pd.read_csv(time_series_df_path)
        analysis_df = analysis_df.append(df)

In [50]:
train_df = analysis_df
test_df = pd.read_csv(PROCESSED_DIR / "Lene_202110222140_scanExport_6027.csv")


In [51]:
def preprocess_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.rename(columns={"Date":"ds", "CountOfPersons":"y"})
    df["ds"] = pd.to_datetime(df.ds)
    return df

In [52]:
train_df = preprocess_df(train_df)
test_df = preprocess_df(test_df)

In [53]:
train_df.head()

,ds,y
0,2020-01-29 16:00:00,1
1,2020-01-29 16:05:00,1
2,2020-01-29 16:10:00,0
3,2020-01-29 16:15:00,0
4,2020-01-29 16:20:00,0


In [114]:
example_df

,ds,y
0,2019-05-19 10:50:00,2
1,2019-05-19 10:55:00,0
2,2019-05-19 11:00:00,0
3,2019-05-19 11:05:00,0
4,2019-05-19 11:10:00,0
...,...,...
64,2019-05-19 16:10:00,2
65,2019-05-19 16:15:00,0
66,2019-05-19 16:20:00,1
67,2019-05-19 16:25:00,0


In [130]:
start_time = '2019-05-19 15:30:00'

In [134]:
str(pd.to_datetime(start_time) - pd.Timedelta("2H"))

'2019-05-19 13:30:00'

In [141]:
def create_sample(df: pd.DataFrame, start_time: str) -> pd.DataFrame:
    opening_time = str(pd.to_datetime(start_time) - pd.Timedelta("2H"))
    df.set_index("ds").query(f"'{opening_time}' <= index <= '{start_time}'").values
    return df.y.values

In [142]:
create_sample(example_df,start_time='2019-05-19 15:30:00')

array([  2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   1,   0,   0,   0,   0,   1,   0,
         0,   0,   0,   0,   0,   0,  31,  26,  22,  39,  30,  72,  87,
       149, 185, 225, 231, 404, 501, 483, 624, 528, 562, 647, 610, 346,
       265, 177, 105,  45,  22,  33,  17,   4,   3,   1,   0,   0,   2,
         0,   1,   0,   2])

In [121]:
example_df.set_index("ds").query("'2019-05-19 13:30:00' <= index <= '2019-05-19 15:30:00'").values

array([[ 31],
       [ 26],
       [ 22],
       [ 39],
       [ 30],
       [ 72],
       [ 87],
       [149],
       [185],
       [225],
       [231],
       [404],
       [501],
       [483],
       [624],
       [528],
       [562],
       [647],
       [610],
       [346],
       [265],
       [177],
       [105],
       [ 45],
       [ 22]])